In [91]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import time
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys_jkeme import g_key

In [92]:
# Load crime data set into pandas
cities_df = pd.read_csv("../WeatherPy/cities_df.csv")
cities_df.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
0,Aklavik,CA,68.2191,-135.0107,1.650735e+09,9.22,1013.0,63.0,23.00,0.0
1,Mount Gambier,AU,-37.8333,140.7667,1.650735e+09,5.75,1032.0,93.0,46.31,0.0
2,Niquelândia,BR,-14.4739,-48.4597,1.650735e+09,5.03,1011.0,38.0,89.44,84.0
3,Coquimbo,CL,-29.9533,-71.3436,1.650735e+09,13.80,1013.0,88.0,58.73,75.0
4,Port Elizabeth,ZA,-33.9180,25.5701,1.650735e+09,3.44,1016.0,96.0,62.31,75.0


In [93]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622 entries, 0 to 621
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         622 non-null    object 
 1   Country      568 non-null    object 
 2   Latitude     573 non-null    float64
 3   Longitude    573 non-null    float64
 4   Date         573 non-null    float64
 5   Wind_Speed   573 non-null    float64
 6   Pressure     573 non-null    float64
 7   Humidity     573 non-null    float64
 8   Temperature  573 non-null    float64
 9   Cloudiness   573 non-null    float64
dtypes: float64(8), object(2)
memory usage: 48.7+ KB


In [94]:
# Drop all the null rows
cities_df_sub = cities_df.dropna().reset_index(drop=True)
cities_df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         568 non-null    object 
 1   Country      568 non-null    object 
 2   Latitude     568 non-null    float64
 3   Longitude    568 non-null    float64
 4   Date         568 non-null    float64
 5   Wind_Speed   568 non-null    float64
 6   Pressure     568 non-null    float64
 7   Humidity     568 non-null    float64
 8   Temperature  568 non-null    float64
 9   Cloudiness   568 non-null    float64
dtypes: float64(8), object(2)
memory usage: 44.5+ KB


In [95]:
# To return a description of the new dataframe
cities_df_sub.describe()

,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
count,568.000000,568.000000,5.680000e+02,568.000000,568.000000,568.000000,568.000000,568.000000
mean,20.919694,19.515069,1.650735e+09,8.066532,1012.897887,69.156690,61.831268,59.609155
std,32.850904,87.283742,2.702839e+02,5.784387,8.005518,23.599355,22.533877,37.679406
min,-54.800000,-175.200000,1.650734e+09,0.000000,977.000000,6.000000,-12.870000,0.000000
25%,-6.098800,-54.716500,1.650735e+09,4.000000,1008.000000,55.000000,48.142500,21.000000
50%,25.332600,24.943250,1.650735e+09,6.870000,1011.000000,74.000000,66.180000,74.500000
75%,48.945500,89.208375,1.650736e+09,11.057500,1017.000000,88.000000,79.287500,99.000000
max,78.218600,179.316700,1.650736e+09,41.380000,1034.000000,100.000000,106.700000,100.000000


# Humidity Heatmap

In [96]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [97]:
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

locations = cities_df_sub.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=cities_df_sub['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria¶
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [98]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness

mask1 = (cities_df_sub.Temperature < 80) & (cities_df_sub.Temperature > 70) & (cities_df_sub.Wind_Speed < 10) & (cities_df_sub.Cloudiness == 0)

good_cities_df = cities_df_sub.loc[mask1]
good_cities_df.head()

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness
142,Elizabeth City,US,36.2946,-76.2511,1.650735e+09,8.05,1028.0,45.0,78.03,0.0
192,Baghdad,IQ,33.3406,44.4009,1.650735e+09,4.61,1008.0,18.0,78.71,0.0
219,Tataouine,TN,32.9297,10.4518,1.650735e+09,2.48,1009.0,12.0,77.63,0.0
285,Bay City,US,43.5945,-83.8889,1.650735e+09,9.22,1017.0,50.0,73.72,0.0
331,Havelock,US,34.8791,-76.9013,1.650735e+09,3.00,1027.0,66.0,78.98,0.0


In [99]:
good_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 142 to 331
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         5 non-null      object 
 1   Country      5 non-null      object 
 2   Latitude     5 non-null      float64
 3   Longitude    5 non-null      float64
 4   Date         5 non-null      float64
 5   Wind_Speed   5 non-null      float64
 6   Pressure     5 non-null      float64
 7   Humidity     5 non-null      float64
 8   Temperature  5 non-null      float64
 9   Cloudiness   5 non-null      float64
dtypes: float64(8), object(2)
memory usage: 440.0+ bytes


# Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [100]:
# geocoordinates test
# Booth office hours help
target_coordinates = f"{-22.9661}, {-42.0278}"
target_search = ""

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Pousada Porto Praia
Rua Vilas Bôas, 161 - Macedônia, Arraial do Cabo
4.3


In [101]:
# exactly like in #Day6-Python-APIs-3-Activities-05-Stu_Google_Complex solved
Hotel_df = good_cities_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# 
for index, row in good_cities_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    places_data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        Hotel_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\jonwukeme\Anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\jonwukeme\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [102]:
Hotel_df

,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness,Hotel Name
142,Elizabeth City,US,36.2946,-76.2511,1.650735e+09,8.05,1028.0,45.0,78.03,0.0,Hampton Inn Elizabeth City
192,Baghdad,IQ,33.3406,44.4009,1.650735e+09,4.61,1008.0,18.0,78.71,0.0,Baghdad Hotel
219,Tataouine,TN,32.9297,10.4518,1.650735e+09,2.48,1009.0,12.0,77.63,0.0,Sangho Privilege Tataouine
285,Bay City,US,43.5945,-83.8889,1.650735e+09,9.22,1017.0,50.0,73.72,0.0,DoubleTree by Hilton Hotel Bay City - Riverfront
331,Havelock,US,34.8791,-76.9013,1.650735e+09,3.00,1027.0,66.0,78.98,0.0,Sherwood Motel


In [103]:
# Create a new dataframe with index reset.
Hotel_df_fin = Hotel_df.reset_index()

In [104]:
# Dataframe with Reset index
Hotel_df_fin

,index,City,Country,Latitude,Longitude,Date,Wind_Speed,Pressure,Humidity,Temperature,Cloudiness,Hotel Name
0,142,Elizabeth City,US,36.2946,-76.2511,1.650735e+09,8.05,1028.0,45.0,78.03,0.0,Hampton Inn Elizabeth City
1,192,Baghdad,IQ,33.3406,44.4009,1.650735e+09,4.61,1008.0,18.0,78.71,0.0,Baghdad Hotel
2,219,Tataouine,TN,32.9297,10.4518,1.650735e+09,2.48,1009.0,12.0,77.63,0.0,Sangho Privilege Tataouine
3,285,Bay City,US,43.5945,-83.8889,1.650735e+09,9.22,1017.0,50.0,73.72,0.0,DoubleTree by Hilton Hotel Bay City - Riverfront
4,331,Havelock,US,34.8791,-76.9013,1.650735e+09,3.00,1027.0,66.0,78.98,0.0,Sherwood Motel


In [105]:
# Booth office hours
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
Hotel_Data = [info_box_template.format(**row) for index, row in Hotel_df_fin.iterrows()]
Locations_Fin = Hotel_df_fin[["Latitude", "Longitude"]]

In [106]:
# Booth office hours
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(Locations_Fin, info_box_content=Hotel_Data)

# Display map
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [107]:
# Add marker layer on top of heat map
marker_layer = gmaps.marker_layer(Locations_Fin, info_box_content=Hotel_Data)
heatmap_layer = gmaps.heatmap_layer(locations, weights=cities_df_sub['Humidity'], max_intensity=20, point_radius=5)


# Display map
fig = gmaps.figure()
# Display map with with both layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))